# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [50]:
def priest(sex, age, respiratRt, OxySatu, heartRt, systolicbp, temper, alertvalue, inOxy, performcSts):
    """

    (String, float, float, float, float, float, float, String, String, String) -> float)

    The priest method is the method used to determine the percentage of the risk
    of the clinical Severity Score (CRSC) 'PRIEST' COVID-19.

    Priest function accepts sex, alertvalue, inspOxy and performcSts in String and remaining all other parameters are in float.

    >>> priest("MALE", 18,23,0.93,89,115,35.1,"confused or not alert","AIR","limited activity, can self-care")
    0.34

    >>> priest("FEMALE",65,23,0.96,105,115,40.8,"ALERT","AIR","limited activity, can self-care")
    0.26

    >>> priest('male', 50, 20, 0.90, 87, 95, 35, 'alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
    0.59
    """

    PRIESTScore = 0

    if sex.upper()=="MALE":
        PRIESTScore += 1

    if age >= 50 and age<=65:
        PRIESTScore = PRIESTScore + 2
    elif age >= 66 and age<=80:
        PRIESTScore = PRIESTScore + 3
    elif age > 80:
        PRIESTScore = PRIESTScore + 4

    if respiratRt > 24:
        PRIESTScore = PRIESTScore +  3
    elif respiratRt >= 21 and respiratRt <= 24:
        PRIESTScore = PRIESTScore +  2
    elif respiratRt >= 9 and respiratRt <= 11:
        PRIESTScore = PRIESTScore +  1
    elif respiratRt < 9:
        PRIESTScore = PRIESTScore +  3

    if OxySatu < 0.92:
        PRIESTScore = PRIESTScore +  3
    elif OxySatu >= 0.94 and OxySatu <= 0.95:
        PRIESTScore = PRIESTScore +  1
    elif OxySatu >= 0.92 and OxySatu <= 0.93:
        PRIESTScore = PRIESTScore +  2

    if heartRt > 130:
        PRIESTScore = PRIESTScore +  3
    elif heartRt >= 111 and heartRt <= 130:
        PRIESTScore = PRIESTScore +  2
    elif heartRt >= 91 and heartRt <= 110:
        PRIESTScore = PRIESTScore +  1
    elif heartRt < 41:
        PRIESTScore = PRIESTScore +  3
    elif heartRt >= 41 and heartRt <= 50:
        PRIESTScore = PRIESTScore +  1

    if systolicbp > 219:
        PRIESTScore = PRIESTScore +  3
    elif systolicbp >= 101 and systolicbp <= 110:
        PRIESTScore = PRIESTScore +  1
    elif systolicbp >= 91 and systolicbp <= 100:
        PRIESTScore = PRIESTScore +  2
    elif systolicbp < 91:
        PRIESTScore = PRIESTScore +  3

    if temper > 39.0:
        PRIESTScore = PRIESTScore +  2
    elif temper >= 38.1 and temper <= 39.0:
        PRIESTScore = PRIESTScore +  1
    if temper < 35.1:
        PRIESTScore = PRIESTScore +  3
    elif temper >= 35.1 and temper <= 36.0:
        PRIESTScore = PRIESTScore +  1

    if alertvalue.lower() == "confused or not alert":
        PRIESTScore = PRIESTScore +  3

    if performcSts.lower() == "limited self-care":
        PRIESTScore = PRIESTScore +  3
    elif performcSts.lower() == "limited activity, can self-care":
        PRIESTScore = PRIESTScore +  2
    if performcSts.lower() == "limited strenuous activity, can do light activity":
        PRIESTScore = PRIESTScore +  1
    elif performcSts.lower() == "bed/chair bound, no self-care":
        PRIESTScore = PRIESTScore +  4

    if inOxy == "supplemental oxygen":
        PRIESTScore = PRIESTScore +  2

    riskPer = [0.01, 0.01, 0.02, 0.02, 0.03, 0.09, 0.15, 0.18, 0.22, 0.26, 0.29, 0.34, 0.38, 0.46, 0.47, 0.49, 0.55, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,0.99]
    PRIESTScore = min(PRIESTScore, len(riskPer) - 1)
    return riskPer[PRIESTScore]

In [51]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest("MALE", 18,23,0.93,89,115,35.1,"confused or not alert","AIR","limited activity, can self-care")
Expecting:
    0.34
ok
Trying:
    priest("FEMALE",65,23,0.96,105,115,40.8,"ALERT","AIR","limited activity, can self-care")
Expecting:
    0.26
ok
Trying:
    priest('male', 50, 20, 0.90, 87, 95, 35, 'alert', 'supplemental oxygen', 'bed/chair bound, no self-care')
Expecting:
    0.59
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [43]:
import requests as req
import json as js
import csv as c


def find_hospital(age, sex, riskPerc):
    """
    (integer, String, float) -> String

    Find_hospital function Use the age, sex and risk percentage for finding a hospital. It
    expects age, risk percentage in float and sex in String and produces hospital name in String.

     find_hospital function find hospital on API by passing age, sex and rper as query parameters to the url
     using requests module


    >>> find_hospital(18, "female", 0.15)
    'Select Specialty Hospital - Northeast Atlanta'

    >>> find_hospital(66, "female", 0.18)
    'Wesley Woods Geriatric Hospital'

    >>> find_hospital(85, "Female", 0.29)
    'Wesley Woods Geriatric Hospital'

    """
    sex = sex.lower()

    urlll = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/"

    data = req.get(urlll, params={"age": age, "sex": sex, "risk_pct": riskPerc})

    return (data.json())['hospital']

In [44]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(18, "female", 0.15)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(66, "female", 0.18)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(85, "Female", 0.29)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [45]:
import requests as req
import json as js

def get_address(hospName):
    """
    (String) -> String

    get_address function find the address of hospital based on hospital name from the url provided

    get_address function expects hospital name as String and return address in String

    get_address function find address by use API, where it has hospital name json data in it.

    >>> get_address("Wesley Woods Geriatric Hospital")
    '1821 CLIFTON ROAD, NE'

    >>> get_address("Select Specialty Hospital - Northeast Atlanta")
    '1821 CLIFTON ROAD NE'

    >>> get_address("Southwest Hospital and Medical Center")
    '501 FAIRBURN ROAD SW'

    """

    httpurll = "https://hds5210-data.s3.amazonaws.com/ga_hospitals.json"

    re = req.get(httpurll)

    hospName=hospName.upper()

    if re.status_code==200:

        ress = re.json()

        if hospName in ress:

            return ress[hospName]["ADDRESS"]
        else:
            return " "

    else:
        print("URL Not WORKING")

In [46]:
import doctest
doctest.run_docstring_examples(get_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    get_address("Wesley Woods Geriatric Hospital")
Expecting:
    '1821 CLIFTON ROAD, NE'
ok
Trying:
    get_address("Select Specialty Hospital - Northeast Atlanta")
Expecting:
    '1821 CLIFTON ROAD NE'
ok
Trying:
    get_address("Southwest Hospital and Medical Center")
Expecting:
    '501 FAIRBURN ROAD SW'
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [47]:
import csv
def process_people(fileURL):
    """
    (string) -> dictionary

    This process_people method will produce all patient data,
    including risk, hospital, and hospital address, in a dictionary.

    process_people accepts input parameter as url which is string and generate dictionary
    with patient info as output
    """
    try:
        finalDataa = {}
        res = req.get(fileURL)
        if res.status_code == 200:
            data = res.text.split("\n")
            dataa = csv.reader(data, delimiter='|')
            next(dataa)
            for row in dataa:
                if all(field == '' for field in row):
                    break

                patient, sex, age, respiratRt, OxySatu, heartRt, systolicbp, temper, alertvalue, inOxy, performcSts = row

                risk = priest(sex, float(age), float(respiratRt), float(OxySatu), float(heartRt), float(systolicbp), float(temper), alertvalue, inOxy, performcSts)

                hosName = find_hospital(int(age), sex, risk)

                addss = get_address(hosName)

                lisst = [sex, float(age), float(respiratRt), float(OxySatu), float(heartRt), float(systolicbp), float(temper), alertvalue, inOxy, performcSts, risk, hosName, addss]

                finalDataa[patient] = lisst
#         print(json_data)


            return finalDataa

        else:
            return " "
    except Exception as err:
        print("ERROR OCCURED WHILE ACCESSING LINK")
        return " "

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [52]:
import requests as req
import json as js

"""
It compares the json data from the link with the calculated data using above functions

It returns "Data is equal" when they are equal and not equal if they are not
"""

url = "https://hds5210-data.s3.amazonaws.com/people_results.json"
headers = {'Accept' : 'application/json'}

try:
    exp= req.get(url, headers=headers)
    if exp.status_code == 200:
        first = exp.json()
        second = process_people("https://hds5210-data.s3.amazonaws.com/people.psv")
#     print("9999",second)

        firstString1 = json.dumps(first)
        secondString1 = json.dumps(second)

        firstString2 = json.loads(firstString1)
        secondString2 = json.loads(secondString1)

    if firstString2 == secondString2:
        print("Data is equal")
    else:
        print("Data is Not equal")


except Exception as err:
  print("Error in The Link")

Error in The Link


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---